In [13]:
# qa_system.py

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import json
import os

# ======= CONFIGURAÇÃO INICIAL =======
load_dotenv()

# Carrega configurações externas
def load_settings():
    with open("../configs/settings.json", "r") as f:
        return json.load(f)

settings = load_settings()
openai_api_key = os.getenv("OPENAI_API_KEY") or settings.get("openai_api_key")
vectorstore_dir = os.path.abspath("../vectorstore")
collection_name = "documents_dados_bcb"

if not openai_api_key:
    raise ValueError("Chave da API OpenAI não encontrada.")

# ======= EMBEDDING E VECTORSTORE =======
def load_vectorstore():
    print(f"📁 Carregando base vetorial de: {vectorstore_dir}")
    embedding = OpenAIEmbeddings(api_key=openai_api_key)
    
    try:
        vectorstore = Chroma(
            collection_name=collection_name,
            persist_directory=vectorstore_dir,
            embedding_function=embedding
        )
        print("✅ Base vetorial carregada com sucesso.")
        return vectorstore
    except Exception as e:
        raise RuntimeError(f"Erro ao carregar vectorstore: {e}")

# ======= PROMPT TEMPLATE =======
prompt_template = PromptTemplate(
    template="""Com base no contexto fornecido, responda à pergunta de forma precisa e detalhada.

Contexto:
{context}

Pergunta: {question}

Responda de forma clara e específica, citando dados quando disponíveis. Se não encontrar a informação no contexto, informe isso.

Resposta:""",
    input_variables=["context", "question"]
)

# ======= INICIALIZAÇÃO DO SISTEMA QA =======
def init_qa_system():
    print("🤖 Inicializando sistema Q&A...")
    
    vectorstore = load_vectorstore()

    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k":10}
    )

    llm = ChatOpenAI(model="gpt-4o", temperature=0, api_key=openai_api_key)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt_template},
        return_source_documents=True
    )

    return qa_chain, vectorstore

# ======= FUNÇÃO DE PERGUNTA =======
def ask_question(question, qa_chain, vectorstore):
    print(f"\n🔎 Pergunta: {question}")
    print("-" * 50)
    
    try:
        # Busca direta para avaliar score
        direct_results = vectorstore.similarity_search_with_score(question, k=3)
        print(f"🔍 Busca direta encontrou {len(direct_results)} resultados")

        if direct_results:
            best_score = direct_results[0][1]
            relevance = "🟢 Alta" if best_score < 0.8 else "🟡 Média" if best_score < 1.2 else "🔴 Baixa"
            print(f"📊 Melhor score: {best_score:.3f} - Relevância: {relevance}")

        # Executa o QA chain
        response = qa_chain({"query": question})
        print(f"\n💬 Resposta:\n{response['result']}")
        print(f"\n📚 Baseado em {len(response['source_documents'])} documento(s):")

        for i, doc in enumerate(response['source_documents'][:3]):
            print(f"\n📄 Fonte {i+1}:\n{doc.page_content[:300]}...")

        return response

    except Exception as e:
        print(f"❌ Erro ao processar pergunta: {e}")
        return None

# ======= TESTE INICIAL (main) =======
if __name__ == "__main__":
    print("=" * 60)
    print("SISTEMA Q&A FUNCIONAL - PRONTO PARA USO!")
    print("=" * 60)

    qa_chain, vectorstore = init_qa_system()

    test_questions = [
        "Qual era o valor da série em dezembro de 2002?",
        "Quais dados estão disponíveis para dezembro de 2002?",
        "Há informações sobre séries temporais em 2002?",
        "Que tipo de dados financeiros estão disponíveis?"
    ]

    for question in test_questions:
        response = ask_question(question, qa_chain, vectorstore)
        print("\n" + "─" * 60)

        if response:
            if "não encontrei" in response['result'].lower() or "não está disponível" in response['result'].lower():
                print("⚠️ Informação não encontrada - pode precisar verificar os dados originais")
            else:
                print("✅ Resposta obtida com sucesso")

    print("\n✨ Testes concluídos!")
    print("Use a função `ask_question('sua pergunta', qa_chain, vectorstore)` para novas consultas.")


SISTEMA Q&A FUNCIONAL - PRONTO PARA USO!
🤖 Inicializando sistema Q&A...
📁 Carregando base vetorial de: /home/edu/Documentos/Engenharia-dados-IA/Pipe-Rag/vectorstore
✅ Base vetorial carregada com sucesso.

🔎 Pergunta: Qual era o valor da série em dezembro de 2002?
--------------------------------------------------

🔎 Pergunta: Qual era o valor da série em dezembro de 2002?
--------------------------------------------------
🔍 Busca direta encontrou 3 resultados
📊 Melhor score: 0.210 - Relevância: 🟢 Alta
🔍 Busca direta encontrou 3 resultados
📊 Melhor score: 0.210 - Relevância: 🟢 Alta

💬 Resposta:
O valor da série em dezembro de 2002 era R$ 477899,39.

📚 Baseado em 10 documento(s):

📄 Fonte 1:
data: 2002-12-01
valor: 47789939
contexto: No mês de dezembro de 2002, o valor da série era R$ 477899,39....

📄 Fonte 2:
data: 1992-12-01
valor: 40373603409
contexto: No mês de dezembro de 1992, o valor da série era R$ 403736034,09....

📄 Fonte 3:
data: 2006-12-01
valor: 82881420
contexto: No mês de 